In [ ]:
import numpy as np
import pandas as pd
import pydeck as pdk
import matplotlib.pyplot as plt

df = pd.read_csv("traffy_flood.csv")

In [ ]:
df

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    get_position=["longitude", "latitude"],
    get_radius=200,
    get_fill_color=[255, 140, 0],
    opacity=0.6,
    pickable=True
)

# Set the viewport location
view_state = pdk.data_utils.compute_view(df[["longitude", "latitude"]])
view_state.zoom = 10

# Render
deck = pdk.Deck(layers=[layer], initial_view_state=view_state, 
                tooltip={"text": "{subdistrict} {district}\n{timestamp}\n{comment}"})
deck.to_html("pydeck_traffy.html")

In [ ]:
# Define the heatmap layer
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    df,
    get_position="[longitude, latitude]",
    opacity=0.5,
    pickable=True
)

view_state = pdk.ViewState(
    latitude=df['latitude'].mean(),
    longitude=df['longitude'].mean(),
    zoom=10
)
pdk.Deck(layers=[heatmap_layer], initial_view_state=view_state)

In [ ]:

pdk.Deck(layers=[layer, heatmap_layer], initial_view_state=view_state)


In [ ]:
# DBSCAN clustering
from sklearn.cluster import DBSCAN

coords = df[['latitude', 'longitude']]
db = DBSCAN(eps=0.005, min_samples=10).fit(coords)
df['cluster'] = db.labels_

# Filter out noise points
df = df[df['cluster'] != -1].copy()

# Count the number of points in each cluster and identify the largest clusters
clusters_count = df['cluster'].value_counts()

# Exclude the '-1' cluster, which represents noise
clusters_count = clusters_count[clusters_count.index != -1]

unique_clusters = df['cluster'].unique()
num_clusters = len(unique_clusters)

# Use a continuous colormap to generate colors, ensure we have enough colors for all clusters.
colormap = plt.get_cmap('hsv')
cluster_colors = {cluster: [int(x*255) for x in colormap(i/num_clusters)[:3]]
                      for i, cluster in enumerate(unique_clusters)}
    
# Map cluster ID to color for each row in the dataframe
df['color'] = df['cluster'].map(cluster_colors)

# Define the scatter plot layer
scatter_layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    get_position="[longitude, latitude]",
    get_color='color',
    get_radius=200,
    opacity=0.5,
    pickable=True
)

view_state = pdk.ViewState(
    latitude=df['latitude'].mean(),
    longitude=df['longitude'].mean(),
    zoom=10
)
pdk.Deck(layers=[scatter_layer], initial_view_state=view_state, tooltip={"text": "{cluster}\n{subdistrict} {district}\n{timestamp}"})


In [ ]:
import matplotlib.pyplot as plt

# Plotting the data
clusters_count.plot(kind='bar', color='blue')  # You can customize the color

plt.xticks(fontsize=8)

# Optional: adjust figure size if labels still overlap
plt.gcf().set_size_inches(12, 6)  # Adjust the size as needed

plt.xlabel('Cluster')  # Set x-axis label, if needed
plt.ylabel('Count')    # Set y-axis label
plt.title('Size of Clusters')  # Set title
plt.show()



In [ ]:
# Kernel Density Estimation Analysis
from sklearn.neighbors import KernelDensity

df = pd.read_csv("traffy_flood.csv")
coords = df[['latitude', 'longitude']]

# Fit KDE model
# Bandwidth controls smoothing - smaller = more detailed, larger = smoother
kde = KernelDensity(bandwidth=0.01, kernel='gaussian')
kde.fit(coords)

# Calculate density score for each point
log_density = kde.score_samples(coords)
density = np.exp(log_density)

# Add density scores to dataframe
df['density'] = density

# Normalize density for better visualization (0-1 range)
df['density_normalized'] = (df['density'] - df['density'].min()) / (df['density'].max() - df['density'].min())

print(f"Density range: {df['density'].min():.2f} to {df['density'].max():.2f}")
print(f"Mean density: {df['density'].mean():.2f}")

In [ ]:
# Visualize density distribution
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Histogram of density values
plt.subplot(1, 2, 1)
plt.hist(df['density'], bins=50, color='teal', alpha=0.7)
plt.xlabel('Density Score')
plt.ylabel('Frequency')
plt.title('Distribution of Density Scores')

# Box plot
plt.subplot(1, 2, 2)
plt.boxplot(df['density'], vert=True)
plt.ylabel('Density Score')
plt.title('Density Score Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Create color mapping based on density (red = high density, blue = low density)
def density_to_color(density_normalized):
    """Convert normalized density to RGB color (blue to red gradient)"""
    # Blue (low) to Red (high) through green
    r = int(density_normalized * 255)
    g = int((1 - abs(2 * density_normalized - 1)) * 255)
    b = int((1 - density_normalized) * 255)
    return [r, g, b]

# Apply color mapping
df['density_color'] = df['density_normalized'].apply(density_to_color)

# Define the scatter plot layer with density-based colors
kde_scatter_layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    get_position="[longitude, latitude]",
    get_color='density_color',
    get_radius=200,
    opacity=0.6,
    pickable=True
)

view_state = pdk.ViewState(
    latitude=df['latitude'].mean(),
    longitude=df['longitude'].mean(),
    zoom=10
)

pdk.Deck(
    layers=[kde_scatter_layer], 
    initial_view_state=view_state, 
    tooltip={
        "text": "Density: {density}\n{subdistrict} {district}\n{timestamp}"
    }
)

In [ ]:
# Show top high-density locations
print("\nTop 10 highest density locations:")
print(df.nlargest(10, 'density')[['subdistrict', 'district', 'density', 'timestamp']])